In [1]:
from pulp import *
import pandas as pd
import numpy as np

In [2]:
#Variables name and costs
variable_names = ["1","2","3","4","5","6","7","8","9","io"]
price=np.array([2,4,2,2,1,2,7,2,2,2])

In [3]:
#Modell
model = LpProblem("Ütemezési feladat", LpMinimize)

#változók: Minden lépésre (sorszám a feladatban szereplés alapján) rendelünk egy kezdő és befejezési értéket, amik különbségét meghatározzuk
allocation=np.array(LpVariable.matrix("X", [i+e for i in ["S_","F_"] for e in variable_names],cat="Integer", lowBound=0))
# Ütemezési feltételek, mi
for i in range(len(price)):
    model += allocation[i] + price[i] <= allocation[i+len(price)]
    model += allocation[i] + price[i] >= allocation[i+len(price)]
    

#Ütemezési feltételek, ahol megadjuke a kezdő és végértékek relációját, mi mi után kell jöjjön
model += allocation[1]>=allocation[0+len(price)]  , "hagyma_pucol-felvág"
model += allocation[3]>=allocation[1+len(price)]  , "hagyma_felvág-párolás"
model += allocation[3]>=allocation[2+len(price)]  , "olaj-párolás"
model += allocation[5]>=allocation[4+len(price)]  , "tojás_tör-felver"
model += allocation[6]>=allocation[3+len(price)]  , "párol-tojássüt"
model += allocation[6]>=allocation[5+len(price)]  , "tojásver-tojássüt"


C:\Users\Csoni\AppData\Roaming\Python\Python39\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [4]:
#Megoldás keresés és eredmények kiiratása
list_of_end_times=[]
pairs=[]
for i in range(10):
    model += allocation[i+len(price)]
    model.solve()
    list_of_end_times.append(model.variables()[i].value())
    pairs.append([model.variables()[i].value(),i])
print("Az optimális ütemezési idővel ennyi időre van szükség: "+str(max(list_of_end_times)))
for i in range(10):
    if list_of_end_times[i]==max(list_of_end_times):
        break
print("Egy optimális ütemezés, ahol az X_F_i az i-edik ferladat befejezési ideje a listában és X_S_i ennek a feladatnak a kezdésideje")
model += allocation[i+len(price)]
model.solve()
for v in model.variables():
    try:
        print(v.name, " = ",v.value())
    except:
        print("error couldnt find value")
    

C:\Users\Csoni\AppData\Roaming\Python\Python39\site-packages\pulp\pulp.py:1711: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Az optimális ütemezési idővel ennyi időre van szükség: 15.0
Egy optimális ütemezés, ahol az X_F_i az i-edik ferladat befejezési ideje a listában és X_S_i ennek a feladatnak a kezdésideje
X_F_1  =  2.0
X_F_2  =  6.0
X_F_3  =  2.0
X_F_4  =  8.0
X_F_5  =  1.0
X_F_6  =  3.0
X_F_7  =  15.0
X_F_8  =  2.0
X_F_9  =  2.0
X_F_io  =  2.0
X_S_1  =  0.0
X_S_2  =  2.0
X_S_3  =  0.0
X_S_4  =  6.0
X_S_5  =  0.0
X_S_6  =  1.0
X_S_7  =  8.0
X_S_8  =  0.0
X_S_9  =  0.0
X_S_io  =  0.0


In [5]:
# Kritikus utat lehetne mélységi kereséssel keresni
# Megnézzük, mik fejeződtek be az utolsó folyamat kezdési idelyében és hogy az feltétele volt-e az utolsó folyamat elkezdésének

# Ehhez először tároljuk a rákövetkezési feltételeket egy tömbben:
feltetelek=[[1,0+len(price)],[3,1+len(price)],[3,2+len(price)],[5,4+len(price)],[6,3+len(price)],[6,5+len(price)]]


lis=[]

def search(model, feltetelek, max_ind, price,lis):
    if int(model.variables()[max_ind+len(price)].value())==0:
        lis.append(max_ind+1)
        return lis
    for j in range(len(price)):
        if model.variables()[j].value()==model.variables()[max_ind+len(price)].value():
            if [max_ind,j+len(price)] in feltetelek:
                lis.append(max_ind+1)
                search(model,feltetelek,j,price,lis)
    return lis

In [6]:
print("Egy kritikus úton lévő feladatok sorszámai:",search(model,feltetelek,i,price,lis))

Egy kritikus úton lévő feladatok sorszámai: [7, 4, 2, 1]
